# Feature Engineering

In [1]:
import os
import pprint
from IPython.display import display, clear_output
from tqdm import tqdm
from collections import defaultdict
from datetime import datetime
import time

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
plt.style.use('seaborn-whitegrid')
import seaborn as sns
sns.set_style('whitegrid')

os.sys.path.append(os.path.abspath(r'../../'))
from configs.config import cfg 

In [2]:
def del_no_answer_data(df: pd.DataFrame, reset_idx: bool = True) -> pd.DataFrame:
    df = df[df.answerCode != -1]
    if reset_idx:
        df.reset_index(drop=True, inplace=True)
    
    return df

## 1. Data Correction

* Column
    * Name 
        * `testId` → `testID`
        * `assessmentItemID` → `assmtID`
        * `Timestamp` → `timestamp`
        * `KnowledgeTag` → `knowledgeTag`
    * New columns
        * `datasetType`
            * 0: Train set
            * 1: Validation set
            * 2: Test set
    * Order
        * [`userID`, `testID`, `assmtID`, `timestamp`, `knowledgeTag`, `answerCode`]

In [3]:
filenames = [r'train_data.csv', r'test_data.csv']
_df_dtype = {
    'userID': np.int16,
    'answerCode': np.int8,
    'KnowledgeTag': np.int16
}

for filename in filenames:
    path = os.path.join(cfg.path.data_dir, r'_ORIGINAL', filename)
    print(filename)

    # Read data.
    df = pd.read_csv(path, dtype=_df_dtype)

    # Correct column names.
    df.rename(
        columns = {
            'testId': 'testID',
            'assessmentItemID': 'assmtID',
            'Timestamp': 'timestamp',
            'KnowledgeTag': 'knowledgeTag'
        },
        inplace=True
    )
    
    # Add new columns.
    if filename == r'train_data.csv':
        df['datasetType'] = 0
    elif filename == r'val_data.csv':
        df['datasetType'] = 1
    elif filename == r'test_data.csv':
        df['datasetType'] = 2
    
    # Sort the column order.
    df = df[['userID', 'testID', 'assmtID', 'timestamp', 'knowledgeTag', 'answerCode', 'datasetType']]

    # Display
    display(df)
    display(df.info(show_counts=True))
    print()
    
    # Write data.
    path = os.path.join(cfg.path.data_dir, filename)
    df.to_csv(path, index=False)

train_data.csv


,userID,testID,assmtID,timestamp,knowledgeTag,answerCode,datasetType
0,0,A060000001,A060001001,2020-03-24 00:17:11,7224,1,0
1,0,A060000001,A060001002,2020-03-24 00:17:14,7225,1,0
2,0,A060000001,A060001003,2020-03-24 00:17:22,7225,1,0
3,0,A060000001,A060001004,2020-03-24 00:17:29,7225,1,0
4,0,A060000001,A060001005,2020-03-24 00:17:36,7225,1,0
...,...,...,...,...,...,...,...
2266581,7441,A030000071,A030071005,2020-06-05 06:50:21,438,0,0
2266582,7441,A040000165,A040165001,2020-08-21 01:06:39,8836,1,0
2266583,7441,A040000165,A040165002,2020-08-21 01:06:50,8836,1,0
2266584,7441,A040000165,A040165003,2020-08-21 01:07:36,8836,1,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2266586 entries, 0 to 2266585
Data columns (total 7 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   userID        2266586 non-null  int16 
 1   testID        2266586 non-null  object
 2   assmtID       2266586 non-null  object
 3   timestamp     2266586 non-null  object
 4   knowledgeTag  2266586 non-null  int16 
 5   answerCode    2266586 non-null  int8  
 6   datasetType   2266586 non-null  int64 
dtypes: int16(2), int64(1), int8(1), object(3)
memory usage: 80.0+ MB


None


test_data.csv


,userID,testID,assmtID,timestamp,knowledgeTag,answerCode,datasetType
0,3,A050000023,A050023001,2020-01-09 10:56:31,2626,1,2
1,3,A050000023,A050023002,2020-01-09 10:56:57,2626,1,2
2,3,A050000023,A050023003,2020-01-09 10:58:31,2625,0,2
3,3,A050000023,A050023004,2020-01-09 10:58:36,2625,0,2
4,3,A050000023,A050023006,2020-01-09 10:58:43,2623,0,2
...,...,...,...,...,...,...,...
260109,7439,A040000130,A040130001,2020-10-14 23:07:23,8832,0,2
260110,7439,A040000130,A040130002,2020-10-14 23:07:41,8832,1,2
260111,7439,A040000130,A040130003,2020-10-14 23:08:02,8244,1,2
260112,7439,A040000130,A040130004,2020-10-14 23:09:31,8244,1,2


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260114 entries, 0 to 260113
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   userID        260114 non-null  int16 
 1   testID        260114 non-null  object
 2   assmtID       260114 non-null  object
 3   timestamp     260114 non-null  object
 4   knowledgeTag  260114 non-null  int16 
 5   answerCode    260114 non-null  int8  
 6   datasetType   260114 non-null  int64 
dtypes: int16(2), int64(1), int8(1), object(3)
memory usage: 9.2+ MB


None

## 2. Test Set을 Validation Set으로 활용

데이터 하나 하나가 정말 소듕하니까...

### 2-1. Concatenate Whole Dataset

In [4]:
filenames = [r'train_data.csv', r'test_data.csv']

df = [pd.read_csv(os.path.join(cfg.path.data_dir, filename), dtype=cfg.data.df_dtype) for filename in filenames]
df = pd.concat(df, ignore_index=True)

display(df)
display(df.info(show_counts=True))
print()
print(f'The number of user IDs: {len(df.userID.unique())}')

,userID,testID,assmtID,timestamp,knowledgeTag,answerCode,datasetType
0,0,A060000001,A060001001,2020-03-24 00:17:11,7224,1,0
1,0,A060000001,A060001002,2020-03-24 00:17:14,7225,1,0
2,0,A060000001,A060001003,2020-03-24 00:17:22,7225,1,0
3,0,A060000001,A060001004,2020-03-24 00:17:29,7225,1,0
4,0,A060000001,A060001005,2020-03-24 00:17:36,7225,1,0
...,...,...,...,...,...,...,...
2526695,7439,A040000130,A040130001,2020-10-14 23:07:23,8832,0,2
2526696,7439,A040000130,A040130002,2020-10-14 23:07:41,8832,1,2
2526697,7439,A040000130,A040130003,2020-10-14 23:08:02,8244,1,2
2526698,7439,A040000130,A040130004,2020-10-14 23:09:31,8244,1,2


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2526700 entries, 0 to 2526699
Data columns (total 7 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   userID        2526700 non-null  int16 
 1   testID        2526700 non-null  object
 2   assmtID       2526700 non-null  object
 3   timestamp     2526700 non-null  object
 4   knowledgeTag  2526700 non-null  int64 
 5   answerCode    2526700 non-null  int8  
 6   datasetType   2526700 non-null  int8  
dtypes: int16(1), int64(1), int8(2), object(3)
memory usage: 86.7+ MB


None


The number of user IDs: 7442


In [5]:
df.to_csv(os.path.join(cfg.path.data_dir, r'data.csv'), index=False)

### 2-2. `answerCode`가 -1인 데이터 제거

학습할 때는 `answerCode`가 -1인 데이터를 제거한 후 사용.

In [6]:
df = del_no_answer_data(df)

display(df)
display(df.info(show_counts=True))

,userID,testID,assmtID,timestamp,knowledgeTag,answerCode,datasetType
0,0,A060000001,A060001001,2020-03-24 00:17:11,7224,1,0
1,0,A060000001,A060001002,2020-03-24 00:17:14,7225,1,0
2,0,A060000001,A060001003,2020-03-24 00:17:22,7225,1,0
3,0,A060000001,A060001004,2020-03-24 00:17:29,7225,1,0
4,0,A060000001,A060001005,2020-03-24 00:17:36,7225,1,0
...,...,...,...,...,...,...,...
2525951,7439,A040000197,A040197006,2020-08-21 07:39:45,2132,1,2
2525952,7439,A040000130,A040130001,2020-10-14 23:07:23,8832,0,2
2525953,7439,A040000130,A040130002,2020-10-14 23:07:41,8832,1,2
2525954,7439,A040000130,A040130003,2020-10-14 23:08:02,8244,1,2


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2525956 entries, 0 to 2525955
Data columns (total 7 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   userID        2525956 non-null  int16 
 1   testID        2525956 non-null  object
 2   assmtID       2525956 non-null  object
 3   timestamp     2525956 non-null  object
 4   knowledgeTag  2525956 non-null  int64 
 5   answerCode    2525956 non-null  int8  
 6   datasetType   2525956 non-null  int8  
dtypes: int16(1), int64(1), int8(2), object(3)
memory usage: 86.7+ MB


None

In [7]:
# df.to_csv(os.path.join(cfg.path.data_dir, r'data_wo_no-answer.csv'), index=False)

## 3. 시간 관련 정보

### 3-1. `unixTime`: Unix time (단위: second)

In [8]:
def cvt_timestamp2unix(timestamp: str) -> int:
    return int(time.mktime(datetime.strptime(timestamp, '%Y-%m-%d %H:%M:%S').timetuple()))

In [9]:
df = pd.read_csv(os.path.join(cfg.path.data_dir, r'data.csv'), dtype=cfg.data.df_dtype)
df.insert(df.columns.get_loc('timestamp') + 1, 'unixTime', df.timestamp.apply(cvt_timestamp2unix))

display(df)
display(df.info(show_counts=True))

,userID,testID,assmtID,timestamp,unixTime,knowledgeTag,answerCode,datasetType
0,0,A060000001,A060001001,2020-03-24 00:17:11,1585009031,7224,1,0
1,0,A060000001,A060001002,2020-03-24 00:17:14,1585009034,7225,1,0
2,0,A060000001,A060001003,2020-03-24 00:17:22,1585009042,7225,1,0
3,0,A060000001,A060001004,2020-03-24 00:17:29,1585009049,7225,1,0
4,0,A060000001,A060001005,2020-03-24 00:17:36,1585009056,7225,1,0
...,...,...,...,...,...,...,...,...
2526695,7439,A040000130,A040130001,2020-10-14 23:07:23,1602716843,8832,0,2
2526696,7439,A040000130,A040130002,2020-10-14 23:07:41,1602716861,8832,1,2
2526697,7439,A040000130,A040130003,2020-10-14 23:08:02,1602716882,8244,1,2
2526698,7439,A040000130,A040130004,2020-10-14 23:09:31,1602716971,8244,1,2


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2526700 entries, 0 to 2526699
Data columns (total 8 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   userID        2526700 non-null  int16 
 1   testID        2526700 non-null  object
 2   assmtID       2526700 non-null  object
 3   timestamp     2526700 non-null  object
 4   unixTime      2526700 non-null  int64 
 5   knowledgeTag  2526700 non-null  int64 
 6   answerCode    2526700 non-null  int8  
 7   datasetType   2526700 non-null  int8  
dtypes: int16(1), int64(2), int8(2), object(3)
memory usage: 106.0+ MB


None

### 3-2. `relTime2Last`: 마지막 문항을 푼 시간으로부터 해당 문항을 푼 상대적인 시각 (단위: second)

마지막으로 푼 문항의 정답 여부를 맞추는 것이 목적이므로, 시간에 대한 정보도 마지막 문항을 푼 시간에 맞춰 상대적으로 주는 것이 좋지 않을까?

※ Sliding window 등의 전처리를 사용할 경우 어차피 다시 계산해야 하므로 미리 계산해놓은 `relTime2Last`는 사용할 수 없다. 학습 직전에 선행되어야 하는 전처리 적용 후 `unixTime` 열의 모든 값에서 마지막 `timestamp`의 `unixTime`을 빼서 사용할 것.

In [10]:
def cvt_unix2rel(group):
    try:
        group.insert(df.columns.get_loc('timestamp') + 2, 'relTime2Last', group.unixTime - group.unixTime.max())    
    except ValueError as err:
        group.loc[:, ['relTime2Last']] = group.unixTime - group.unixTime.max()
        
    return group

In [11]:
grouped = df.groupby('userID')
tqdm.pandas()
df = grouped.progress_apply(cvt_unix2rel)

display(df)
display(df.info(show_counts=True))

100%|██████████| 7442/7442 [00:11<00:00, 649.81it/s] 


,userID,testID,assmtID,timestamp,unixTime,relTime2Last,knowledgeTag,answerCode,datasetType
0,0,A060000001,A060001001,2020-03-24 00:17:11,1585009031,-23685788,7224,1,0
1,0,A060000001,A060001002,2020-03-24 00:17:14,1585009034,-23685785,7225,1,0
2,0,A060000001,A060001003,2020-03-24 00:17:22,1585009042,-23685777,7225,1,0
3,0,A060000001,A060001004,2020-03-24 00:17:29,1585009049,-23685770,7225,1,0
4,0,A060000001,A060001005,2020-03-24 00:17:36,1585009056,-23685763,7225,1,0
...,...,...,...,...,...,...,...,...,...
2526695,7439,A040000130,A040130001,2020-10-14 23:07:23,1602716843,-160,8832,0,2
2526696,7439,A040000130,A040130002,2020-10-14 23:07:41,1602716861,-142,8832,1,2
2526697,7439,A040000130,A040130003,2020-10-14 23:08:02,1602716882,-121,8244,1,2
2526698,7439,A040000130,A040130004,2020-10-14 23:09:31,1602716971,-32,8244,1,2


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2526700 entries, 0 to 2526699
Data columns (total 9 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   userID        2526700 non-null  int16 
 1   testID        2526700 non-null  object
 2   assmtID       2526700 non-null  object
 3   timestamp     2526700 non-null  object
 4   unixTime      2526700 non-null  int64 
 5   relTime2Last  2526700 non-null  int64 
 6   knowledgeTag  2526700 non-null  int64 
 7   answerCode    2526700 non-null  int8  
 8   datasetType   2526700 non-null  int8  
dtypes: int16(1), int64(3), int8(2), object(3)
memory usage: 209.1+ MB


None

In [12]:
df.to_csv(os.path.join(cfg.path.data_dir, r'data.csv'), index=False)

In [13]:
# df = del_no_answer_data(df)
# grouped = df.groupby('userID')
# df = grouped.apply(cvt_unix2rel)

# display(df)
# display(df.info(show_counts=True))

In [14]:
# df.to_csv(os.path.join(cfg.path.data_dir, r'data_wo_no-answer.csv'), index=False)

### 3-3. `elapsedTime`: 문항을 푸는데 걸린 시간 (단위: second)

문제 풀이 시간이:

* 적당했다면 잘 풀었을 가능성이 높다.
* 너무 오래 걸렸다면 해당 문항이 어려워서 그랬을 것이다. vs. 시간을 많이 투자해서 잘 풀었을 수도 있다.
* 너무 조금 걸렸다면 찍거나 막 누르는 등 제대로 안 풀었을 것이다. vs. 그만큼 해당 문항이 쉬웠을 수도 있다.

Outliers 제거

* 각 사용자가 마지막으로 푼 문제는 정확한 풀이 시간을 계산할 수 없다.
  * -1로 맵핑
* `timestamp`가 각 문항을 풀기 시작한 시간이기 때문에 데이터에 노이즈(e.g. 시작을 눌러놓고 한참 뒤에 푸는 경우 등)가 있다.
  * Max elapsed time을 초과하는 경우 비정상적인 데이터로 간주 -2로 맵핑
  * Max elapsed time: 1 hour
* 위 두 가지 경우에 해당하는 데이터는 해당 문항에 대한 풀이 시간 분포의 median 값으로 대체
    * 문제 풀이 시간별 정답률 추이가 1시간까지 점차 감소했다가 그 이후로는 규칙성이 사라지는 경향을 띈다.

In [15]:
def cal_elapsed_time(group):
    group.sort_values(by='unixTime', axis=0, inplace=True)
    
    group.insert(df.columns.get_loc('timestamp') + 3, 'elapsedTime', (group.unixTime.values[1:] - group.unixTime.values[:-1]).tolist() + [-1])
#     mask = (group.testID.values[:-1] != group.testID.values[1:])
#     group.elapsedTime.iloc[:-1][mask] = -1
        
    return group

In [16]:
max_elapsed_time = 60 * 60

df = pd.read_csv(os.path.join(cfg.path.data_dir, r'data.csv'), dtype=cfg.data.df_dtype)

grouped = df.groupby('userID')
tqdm.pandas()
df = grouped.progress_apply(cal_elapsed_time)
df.reset_index(drop=True, inplace=True)
df.loc[df.elapsedTime > max_elapsed_time, ['elapsedTime']] = -2

display(df)
display(df.info(show_counts=True))
display(df.loc[df.elapsedTime >= 0, ['elapsedTime']].describe())

100%|██████████| 7442/7442 [00:15<00:00, 492.58it/s]


,userID,testID,assmtID,timestamp,unixTime,relTime2Last,elapsedTime,knowledgeTag,answerCode,datasetType
0,0,A060000001,A060001001,2020-03-24 00:17:11,1585009031,-23685788,3,7224,1,0
1,0,A060000001,A060001002,2020-03-24 00:17:14,1585009034,-23685785,8,7225,1,0
2,0,A060000001,A060001003,2020-03-24 00:17:22,1585009042,-23685777,7,7225,1,0
3,0,A060000001,A060001004,2020-03-24 00:17:29,1585009049,-23685770,7,7225,1,0
4,0,A060000001,A060001005,2020-03-24 00:17:36,1585009056,-23685763,11,7225,1,0
...,...,...,...,...,...,...,...,...,...,...
2526695,7441,A030000071,A030071005,2020-06-05 06:50:21,1591339821,-6632308,-2,438,0,0
2526696,7441,A040000165,A040165001,2020-08-21 01:06:39,1597971999,-130,11,8836,1,0
2526697,7441,A040000165,A040165002,2020-08-21 01:06:50,1597972010,-119,46,8836,1,0
2526698,7441,A040000165,A040165003,2020-08-21 01:07:36,1597972056,-73,73,8836,1,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2526700 entries, 0 to 2526699
Data columns (total 10 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   userID        2526700 non-null  int16 
 1   testID        2526700 non-null  object
 2   assmtID       2526700 non-null  object
 3   timestamp     2526700 non-null  object
 4   unixTime      2526700 non-null  int64 
 5   relTime2Last  2526700 non-null  int64 
 6   elapsedTime   2526700 non-null  int64 
 7   knowledgeTag  2526700 non-null  int64 
 8   answerCode    2526700 non-null  int8  
 9   datasetType   2526700 non-null  int8  
dtypes: int16(1), int64(4), int8(2), object(3)
memory usage: 144.6+ MB


None

,elapsedTime
count,2.157252e+06
mean,8.842425e+01
std,2.551784e+02
min,0.000000e+00
25%,1.000000e+01
50%,2.800000e+01
75%,6.900000e+01
max,3.600000e+03


In [17]:
def cvt_elapsed_time_outlier2assmt_median(group):
    mask = group.elapsedTime >= 0    
    group.loc[~mask, ['elapsedTime']] = group.loc[mask, 'elapsedTime'].median()
    
    return group

In [18]:
grouped = df.groupby('assmtID')
tqdm.pandas()
df = grouped.progress_apply(cvt_elapsed_time_outlier2assmt_median)
df.reset_index(drop=True, inplace=True)

display(df)
display(df.info(show_counts=True))
display(df[['elapsedTime']].describe())

100%|██████████| 9454/9454 [00:26<00:00, 357.83it/s]


,userID,testID,assmtID,timestamp,unixTime,relTime2Last,elapsedTime,knowledgeTag,answerCode,datasetType
0,0,A060000001,A060001001,2020-03-24 00:17:11,1585009031,-23685788,3.0,7224,1,0
1,0,A060000001,A060001002,2020-03-24 00:17:14,1585009034,-23685785,8.0,7225,1,0
2,0,A060000001,A060001003,2020-03-24 00:17:22,1585009042,-23685777,7.0,7225,1,0
3,0,A060000001,A060001004,2020-03-24 00:17:29,1585009049,-23685770,7.0,7225,1,0
4,0,A060000001,A060001005,2020-03-24 00:17:36,1585009056,-23685763,11.0,7225,1,0
...,...,...,...,...,...,...,...,...,...,...
2526695,7441,A030000071,A030071005,2020-06-05 06:50:21,1591339821,-6632308,401.0,438,0,0
2526696,7441,A040000165,A040165001,2020-08-21 01:06:39,1597971999,-130,11.0,8836,1,0
2526697,7441,A040000165,A040165002,2020-08-21 01:06:50,1597972010,-119,46.0,8836,1,0
2526698,7441,A040000165,A040165003,2020-08-21 01:07:36,1597972056,-73,73.0,8836,1,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2526700 entries, 0 to 2526699
Data columns (total 10 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   userID        2526700 non-null  int16  
 1   testID        2526700 non-null  object 
 2   assmtID       2526700 non-null  object 
 3   timestamp     2526700 non-null  object 
 4   unixTime      2526700 non-null  int64  
 5   relTime2Last  2526700 non-null  int64  
 6   elapsedTime   2526700 non-null  float64
 7   knowledgeTag  2526700 non-null  int64  
 8   answerCode    2526700 non-null  int8   
 9   datasetType   2526700 non-null  int8   
dtypes: float64(1), int16(1), int64(3), int8(2), object(3)
memory usage: 144.6+ MB


None

,elapsedTime
count,2.526700e+06
mean,1.069206e+02
std,2.526852e+02
min,0.000000e+00
25%,1.200000e+01
50%,3.300000e+01
75%,9.900000e+01
max,3.600000e+03


In [19]:
df.to_csv(os.path.join(cfg.path.data_dir, r'data.csv'), index=False)